# Import packages

In [9]:
import pandas as pd
import binance
import ta
from datetime import datetime
import requests

In [10]:
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time
from math import *
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# Define client, download and clean historical data

In [11]:
# Get historical data from Binance
from binance.client import Client
client = Client()

# Define parameters
pairname = "ETHUSDT"
start_date = "10 september 2018"
time_interval = Client.KLINE_INTERVAL_1HOUR

# -- Load all price data from binance API -- #
klines_data = client.get_historical_klines(pairname, time_interval, start_date)
klines_data

# Transform data retrieved from Binance to dataframe // https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data
rawpricedata_df = None
rawpricedata_df = pd.DataFrame(klines_data, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

## CLEAN DATA

del rawpricedata_df['ignore']
del rawpricedata_df['close_time']
del rawpricedata_df['quote_av']
del rawpricedata_df['trades']
del rawpricedata_df['tb_base_av']
del rawpricedata_df['tb_quote_av']

# transform open, close, high, low to numeric
rawpricedata_df['open'] = pd.to_numeric(rawpricedata_df['open'])
rawpricedata_df['close'] = pd.to_numeric(rawpricedata_df['close'])
rawpricedata_df['high'] = pd.to_numeric(rawpricedata_df['high'])
rawpricedata_df['low'] = pd.to_numeric(rawpricedata_df['low'])


## CONVERT TIME and set timestamp as index

rawpricedata_df = rawpricedata_df.set_index(rawpricedata_df['timestamp'])
rawpricedata_df.index = pd.to_datetime(rawpricedata_df.index, unit='ms')
del rawpricedata_df['timestamp']
print("data 100% loaded")
print(rawpricedata_df)


# Create graph to visualize data
import plotly.graph_objects as go

#Create graph object
fig = go.Figure()

#Set up the layout
fig.update_layout(
    title={
        'text':pairname,
        'x': 0.5,
        'xanchor': 'center'
        },
        xaxis_title = "Date",
        yaxis_title = "Price",
        xaxis_rangeslider_visible=False
)

fig.add_trace(
    go.Candlestick(
        x=rawpricedata_df.index,
        open=rawpricedata_df['open'],
        high=rawpricedata_df['high'],
        low=rawpricedata_df['low'],
        close=rawpricedata_df['close']
    )
)
fig.show()


data 100% loaded
                        open     high      low    close          volume
timestamp                                                              
2018-09-10 00:00:00   196.13   198.99   195.95   197.50  10639.76117000
2018-09-10 01:00:00   197.59   203.42   196.95   200.63  24516.67690000
2018-09-10 02:00:00   200.70   201.55   198.49   199.92  17857.00050000
2018-09-10 03:00:00   199.82   199.98   197.61   199.30  18205.58684000
2018-09-10 04:00:00   199.28   199.30   196.00   196.55  17143.02326000
...                      ...      ...      ...      ...             ...
2023-02-22 02:00:00  1645.07  1645.77  1621.59  1644.85  44148.58030000
2023-02-22 03:00:00  1644.85  1651.60  1642.55  1649.25  12081.38530000
2023-02-22 04:00:00  1649.26  1649.97  1637.97  1644.45  13942.37360000
2023-02-22 05:00:00  1644.46  1644.96  1636.14  1642.48  13297.95450000
2023-02-22 06:00:00  1642.47  1644.15  1631.31  1631.95  15027.26730000

[38962 rows x 5 columns]


# Define technical indicators

In [12]:
pricedata_df = None
pricedata_df = rawpricedata_df.copy()

# -- Trix Indicator --
trixLength = 26
trixSignal = 18
pricedata_df['TRIX'] = ta.trend.ema_indicator(ta.trend.ema_indicator(ta.trend.ema_indicator(close=pricedata_df['close'], window=trixLength), window=trixLength), window=trixLength)
pricedata_df['TRIX_PCT'] = pricedata_df["TRIX"].pct_change()*100 # calcule le % de variation P/r a la ligne précédente / example: data = [[10, 18, 11], [20, 15, 8], [30, 20, 3]] df = pd.DataFrame(data) print(df.pct_change())
pricedata_df['TRIX_SIGNAL'] = ta.trend.sma_indicator(pricedata_df['TRIX_PCT'],trixSignal)
pricedata_df['TRIX_HISTO'] = pricedata_df['TRIX_PCT'] - pricedata_df['TRIX_SIGNAL']
# TRIX HISTO : buy signal = TRIX PCT croise TRIX SIGNAL A LA HAUSSE, donc quand TRIX PCT > TRIX SIGNAL, soit TRIX HISTO > 0

# -- Stochasitc RSI --
stochTop = 0.82
stochBottom = 0.2
stochWindow = 10
pricedata_df['STOCH_RSI'] = ta.momentum.stochrsi(close=pricedata_df['close'], window=stochWindow, smooth1=3, smooth2=3)
# stoch RSI parameters (standard)

# Add moving average cross not to trade in downtrends
short_sma = 40 #8 #10 #8 #10
long_sma = 400 #215 #110 #190 #125
pricedata_df['SHORT_SMA'] = ta.trend.sma_indicator(pricedata_df['close'], short_sma)
pricedata_df['LONG_SMA'] = ta.trend.sma_indicator(pricedata_df['close'], long_sma)

# Add AO to spot trend
#aoParam1 = 6
#aoParam2 = 22
#pricedata_df['AO'] = ta.momentum.awesome_oscillator(pricedata_df['high'],pricedata_df['low'],window1=aoParam1,window2=aoParam2)

print("Indicators loaded 100%")

# -- Uncomment the line below if you want to check your dataset with indicators --
pricedata_df

Indicators loaded 100%


,open,high,low,close,volume,TRIX,TRIX_PCT,TRIX_SIGNAL,TRIX_HISTO,STOCH_RSI,SHORT_SMA,LONG_SMA
timestamp,,,,,,,,,,,,
2018-09-10 00:00:00,196.13,198.99,195.95,197.50,10639.76117000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-10 01:00:00,197.59,203.42,196.95,200.63,24516.67690000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-10 02:00:00,200.70,201.55,198.49,199.92,17857.00050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-10 03:00:00,199.82,199.98,197.61,199.30,18205.58684000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-10 04:00:00,199.28,199.30,196.00,196.55,17143.02326000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-22 02:00:00,1645.07,1645.77,1621.59,1644.85,44148.58030000,1692.625957,-0.029859,-0.005882,-0.023977,0.336913,1686.21225,1618.266275
2023-02-22 03:00:00,1644.85,1651.60,1642.55,1649.25,12081.38530000,1692.067671,-0.032983,-0.008518,-0.024466,0.536978,1684.59925,1618.232900
2023-02-22 04:00:00,1649.26,1649.97,1637.97,1644.45,13942.37360000,1691.456909,-0.036096,-0.011249,-0.024847,0.542289,1683.03575,1618.223525


## Optimization

L'objectif est d'optimiser les paramètres trix lenght et trix signal

Trix length = nombre de périodes sur lesquelles on applique la triple moyenne exponentielle

Trix signal = nombre de période de la SMA permettant de créer le signal de trading au croisement avec la triple moyenne exponentielle

In [14]:
# -- Condition to BUY market --
def buyCondition(row):
  if row['TRIX_HISTO'] > 0 and row['STOCH_RSI'] < stochTop: # and row['SHORT_SMA'] > row['LONG_SMA']:
    return True
  else:
    return False

# -- Condition to SELL market --  
def sellCondition(row):
  if row['TRIX_HISTO'] < 0 and row['STOCH_RSI'] > stochBottom:
    return True
  else:
    return False

In [15]:
# -- Define the period on which you want to run the backtest
backtest_start_date = '2019-01-01'
backtest_end_date = '2022-06-01'
test_pricedata_df = None
test_pricedata_df = pricedata_df[backtest_start_date:backtest_end_date].copy()

# Définition de la liste qui sera utilisée pour stocker le résultat de l'optimisation sur les différentes valeurs des paramètres
opti_results = [] # il faudra le transformer en df avec ces colonnes ['parameter1','parameter2', 'result'])

# PARAM1
loopI = [8,30,2]
enumI = ceil((loopI[1] - loopI[0]) / loopI[2]) # c'est le nombre d'itérations total a réaliser (11-7) / 1 = 4
#On ne fait pas +1 car on va initialiser count à 0 et non à 1 !!

# # PARAM 2
loopJ = [16,24,2]  # c'est le nombre d'itérations total a réaliser (15-25) / 1 = 4) 
enumJ = ceil((loopJ[1] - loopJ[0]) / loopJ[2])

# # PARAM 3
loopK = [8,16,2]
enumK = ceil((loopK[1] - loopK[0]) / loopK[2])

# # PARAM 4
loopL = [20,60,10]
enumL = ceil((loopL[1] - loopL[0]) / loopL[2])

# # PARAM 5
loopM = [360,500,20]
enumM = ceil((loopM[1] - loopM[0]) / loopM[2])

count = 0
maxCount = enumI * enumJ * enumK  * enumL * enumM # nombre total d'itérations
print(maxCount)

4928


In [16]:
## LAUNCH OPTIMIZATION
count = 0
opti_results = []

for i in range(loopI[0], loopI[1], loopI[2]):  #(pour i de 7 à 11) -> trix length
   for j in range(loopJ[0], loopJ[1], loopJ[2]):  #(pour j de 15 à 25) -> trix signal
     for k in range(loopK[0], loopK[1], loopK[2]):
       for l in range (loopL[0], loopL[1], loopL[2]) :
         for m in range(loopM[0], loopM[1], loopM[2]) :
      
          clear_output(wait=True)
          count += 1
          print("Loading...",count,'/',maxCount)
          # -- You can change variables below --
          usdt = 1000
          coin = 0
          takerFee = 0.0010

          trixLength = i
          trixSignal = j
          stochWind = k
          shortSMA = 40
          longSMA = 400
          # ici on recalcule le trix en changeant la Length et le Signal à chaque itération !
          test_pricedata_df['TRIX'] = ta.trend.ema_indicator(ta.trend.ema_indicator(ta.trend.ema_indicator(close=test_pricedata_df['close'], window=trixLength), window=trixLength), window=trixLength)
          test_pricedata_df['TRIX_PCT'] = test_pricedata_df["TRIX"].pct_change()*100
          test_pricedata_df['TRIX_SIGNAL'] = ta.trend.sma_indicator(test_pricedata_df['TRIX_PCT'],trixSignal)
          test_pricedata_df['TRIX_HISTO'] = test_pricedata_df['TRIX_PCT'] - test_pricedata_df['TRIX_SIGNAL']

          # recalcul du STOCH RSI
          test_pricedata_df['STOCH_RSI'] = ta.momentum.stochrsi(close=test_pricedata_df['close'], window=stochWind, smooth1=3, smooth2=3)

          # recalcul des MA
          test_pricedata_df['SHORT_SMA'] = ta.trend.sma_indicator(pricedata_df['close'], shortSMA)
          test_pricedata_df['LONG_SMA'] = ta.trend.sma_indicator(pricedata_df['close'], longSMA)

          for index, row in test_pricedata_df.iterrows():
            #BUY
            if buyCondition(row) and usdt > 0:
              coin = (usdt/test_pricedata_df['close'][index]) - takerFee*(usdt/test_pricedata_df['close'][index])
              usdt = 0

            #SELL
            elif sellCondition(row) and coin > 0:
              usdt = coin*test_pricedata_df['close'][index] - (takerFee*coin*test_pricedata_df['close'][index])
              coin = 0
            
          result = coin*test_pricedata_df.iloc[len(test_pricedata_df)-1]['close'] + usdt
          myrow = [i, j, k, l, m, result]
          opti_results.append(myrow)  

Loading... 4928 / 4928


In [19]:
## Build le data frame
opti_results_df = None
opti_results_df = pd.DataFrame (opti_results, columns = ['trix_length', 'trix_signal', 'stoch_window','shortSMA', 'longSMA', 'wallet'])
print(opti_results_df.sort_values(by=['wallet'], ascending=False).head(50))

# Afficher le graphique
# opti_results_df.plot.scatter(x='trix_sig',y='wallet', s=50, colormap='OrRd',figsize=(15,6))
# plt.show()

      trix_length  trix_signal  stoch_window  shortSMA  longSMA        wallet
4189           26           18            10        40      420  18919.616725
4190           26           18            10        40      440  18502.685079
4188           26           18            10        40      400  18190.057008
4217           26           18            12        40      420  17812.537942
4161           26           18             8        40      420  17762.056373
4525           28           16            10        40      420  17241.268779
4497           28           16             8        40      420  17180.316204
4218           26           18            12        40      440  17112.930455
4160           26           18             8        40      400  17077.133363
3069           20           22            10        40      420  16996.102570
4526           28           16            10        40      440  16987.588699
4553           28           16            12        40      420 

In [29]:
opti_results_df.to_csv('final_opti_ETH_trix.csv', encoding='utf-8')